# COVID-19 Interactive Analysis Dashboard

### Import the required libraries

In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
import folium

from IPython.core.display import display,HTML

import ipywidgets as widgets
from ipywidgets import interact, interactive,fixed, interact_manual
import plotly.graph_objects as go

### Loading the datasets

In [3]:
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')

### Changing the column names to lower case

In [4]:
country_df.columns = map(str.lower,country_df.columns)
death_df.columns = map(str.lower,death_df.columns)
recovered_df.columns = map(str.lower,recovered_df.columns)
confirmed_df.columns = map(str.lower,confirmed_df.columns)

### Renaming column names

In [5]:
country_df = country_df.rename(columns={'country_region':'country'})
death_df = death_df.rename(columns={'province/state':'state','country/region':'country'})
confirmed_df = confirmed_df.rename(columns={'province/state':'state','country/region':'country'})
recovered_df = recovered_df.rename(columns={'province/state':'state','country/region':'country'})

### Getting the total count for each case

In [6]:
confirmed_total = int(country_df['confirmed'].sum())
death_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

In [7]:
display(HTML("<div style = 'background-color: #504e4e; padding:30px'>"+
             "<span style='color: white; font-size:20px;'> Confirmed: "  + str(confirmed_total) +"</span>"+
             "<span style='color: red; font-size:20px; margin-left:20px'> Deaths: " + str(death_total)+"</span>"+
             "<span style='color:green; font-size:20px; margin-left:20px'> Recovered: " + str(recovered_total)+"</span>"+
             "<span style='color:yellow; font-size:20px; margin-left:20px'> Active: " + str(active_total)+"</span>"+
             "</div>"

))

## Confirmed/Recovered/Deaths for each country

### Enter the number of countries you want to see the data for:

In [8]:
fig = go.FigureWidget(layout=go.Layout())
def hightlight_col(x):
    r= 'background-color: red'
    y= 'background-color: yellow'
    g= 'background-color: green'
    df1 = pd.DataFrame('',index=x.index, columns=x.columns)
    df1.iloc[:,4] = y
    df1.iloc[:,5] = r
    df1.iloc[:,6] = g
    
    return df1

def show_latest(n):
    n = int(n)
    return country_df.sort_values('confirmed', ascending=False).head(n).style.apply(hightlight_col, axis=None)

interact(show_latest, n='10')

ipywLayout = widgets.Layout(border='solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig],layout=ipywLayout)

interactive(children=(Text(value='10', description='n'), Output()), _dom_classes=('widget-interact',))

    'data': [], 'layout': {'template': '...'}
}),), layout=Layout(border='solid …

## Use the slider to filter and check for the most affected countries

In [9]:
sorted_country_list = country_df.sort_values('confirmed', ascending=False)

In [10]:
def bubble_chart(n):
    fig = px.scatter(sorted_country_list.head(n), x='country', y='confirmed', size='confirmed',color='country', 
                     hover_name='country', size_max=50)
    fig.update_layout(
    title=str(n) + "Worst hit countries",
    xaxis_title = "Countries",
    yaxis_title = "Confirmed Cases",
    width=700
    )
    fig.show()

interact(bubble_chart, n=10)

ipywLayout = widgets.Layout(border = 'solid 2px green')
ipywLayout.display = 'none'
widgets.VBox([fig], layout=ipywLayout)

interactive(children=(IntSlider(value=10, description='n', max=30, min=-10), Output()), _dom_classes=('widget-…

    'data': [], 'layout': {'autosize': True, 'template': '...'}
}),), layout=Lay…

## Check the no. of confirmed and death cases for any country

In [11]:
def plot_cases_country(country):
    labels = ['confirmed','deaths']
    colors = ['blue','red']
    mode_size = [6,8]
    line_size = [4,5]
    
    df_list = [confirmed_df, death_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country == 'World' or country == 'world':
            x_data = np.array(list(df.iloc[:,20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis=0)
        else:
            x_data = np.array(list(df.iloc[:,20:].columns))
            y_data = np.sum(np.asarray(df[df['country']==country].iloc[:,20:]), axis=0)
        
        fig.add_trace(go.Scatter(x=x_data, y=y_data,name=labels[i],
                                line=dict(color=colors[i],width=line_size[i]),
                                connectgaps=True,
                                mode='lines+markers',
                                text="Total"+str(labels[i])+": "+str(y_data[-1])));
        
        fig.update_layout(title="COVID-19 cases of "+ country,
                         xaxis_title='Date',
                         yaxis_title='No. of confirmed cases',
                         margin=dict(l=20,r=20,t=40,b=20),
                         paper_bgcolor='lightgrey',
                         width=800);
        
        fig.update_yaxes(type='linear')
        fig.show();

In [12]:
interact(plot_cases_country, country='world')

ipywLayout = widgets.Layout(border = 'solid 2px green')
ipywLayout.display='none'
widgets.VBox([fig],layout=ipywLayout)

interactive(children=(Text(value='world', description='country'), Output()), _dom_classes=('widget-interact',)…

    'data': [], 'layout': {'autosize': True, 'template': '...'}
}),), layout=Lay…

### Plot showing confirmed cases for Top '10' countries

In [13]:
px.bar(sorted_country_list.head(10),
      x='country',
      y='confirmed',
      title='Top 10 countries with confirmed cases',
      color_discrete_sequence=['pink'],
      height=500,
      width=900)

### Plot showing death cases for Top '10' countries

In [14]:
px.bar(sorted_country_list.head(10),
      x='country',
      y='deaths',
      title='Top 10 countries with deaths',
      color_discrete_sequence=['pink'],
      height=500,
      width=900)

### Plot showing recovered cases for Top '10' countries

In [15]:
px.bar(sorted_country_list.head(10),
      x='country',
      y='recovered',
      title='Top 10 countries by recovered cases',
      color_discrete_sequence=['pink'],
      height=500,
      width=900)

### Plot showing active cases for Top '10' countries

In [16]:
px.bar(sorted_country_list.head(10),
      x='country',
      y='active',
      title='Top 10 countries by active cases',
      color_discrete_sequence=['pink'],
      height=500,
      width=900)

## Viewing Global spread of COVID-19 on the world map

In [17]:
world_map = folium.Map(location=[11,0],tiles='cartodbpositron',zoom_start=2,max_zoom=6,min_zoom=2)

In [18]:
for i in range(0,len(confirmed_df)):
    folium.Circle(
        location=[confirmed_df.iloc[i]['lat'],confirmed_df.iloc[i]['long']],
        fill=True,
        radius=(int((np.log(confirmed_df.iloc[i,-1]+1.00001)))+0.2)*50000,
        color='red',
        fill_color='indigo',
        tooltip="<div style='margin: 0; background-color: black; color: white;'>"+
                    "<h4 style='text-align:center;font-weight: bold'>"+confirmed_df.iloc[i]['country'] + "</h4>"
                    "<hr style='margin:10px;color: white;'>"+
                    "<ul style='color: white;;list-style-type:circle;align-item:left;padding-left:20px;padding-right:20px'>"+
                        "<li>Confirmed: "+str(confirmed_df.iloc[i,-1])+"</li>"+
                        "<li>Deaths:   "+str(death_df.iloc[i,-1])+"</li>"+
                        "<li>Death Rate: "+ str(np.round(death_df.iloc[i,-1]/(confirmed_df.iloc[i,-1]+1.00001)*100,2))+ "</li>"+
                    "</ul></div>").add_to(world_map)

    
world_map